In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mlds

Mounted at /content/drive
/content/drive/MyDrive/mlds


In [ ]:
import data_auto_sys as aus
import tasker_link_prediction as t_lp
from splitter import splitter
from models import Temporal_GCN
from trainer import Trainer
import yaml
import os

In [ ]:
prep = False

with open('config/config_AS_ablation_temporal_n2v.yaml', 'r') as file:
    config = yaml.safe_load(file)

model_path = config['model_path']

if not os.path.exists(model_path):
    os.mkdir(model_path)
    os.mkdir(model_path + "best_checkpoints/")
    os.mkdir(model_path + "latest_checkpoints/")

with open(model_path + 'config.yaml', 'w') as file:
    yaml.dump(config, file)

config

{'adam_config': {'adam_beta_1': 0.9,
  'adam_beta_2': 0.999,
  'adam_epsilon': 1e-07,
  'adam_learning_rate': 0.005},
 'model_path': 'models/AS_Ablation_N2V_Temporal/',
 'classifier_hidden_size': 20,
 'ffn_fusion_size': 10,
 'ffn_hidden_size': 10,
 'gcn_fusion_size': 20,
 'spatial_hidden_size': 20,
 'spatial_input_dim': 128,
 'temporal_hidden_size': 20,
 'temporal_input_dim': 200,
 'num_epochs': 1000,
 'patience': 10,
 'major_threshold': None,
 'train_proportion': 0.7,
 'dev_proportion': 0.1,
 'data_path': 'data/AS-733/',
 'prep_data_path': 'prep_data/AS733_neg/'}

In [ ]:
data = aus.Autonomous_System_Dataset(config['data_path'])
tasker = t_lp.Link_Pred_Tasker(data, path=config['prep_data_path'], prep=prep,
                               embs_dim=config['spatial_input_dim'], temp_dim=int(config['temporal_input_dim']/10),
                               major_threshold=config['major_threshold'], smart_neg_sampling=True, neg_sample=1000)

splitter_as = splitter(tasker, train_proportion = config['train_proportion'], dev_proportion = config['dev_proportion'])

model= Temporal_GCN(temporal_input_dim=config['temporal_input_dim'],
                 temporal_hidden_size=config['temporal_hidden_size'],
                 classifier_hidden_size=config['classifier_hidden_size'],
                 gcn_fusion_size=config['gcn_fusion_size'])

trainer = Trainer(model=model, splitter=splitter_as, model_path=model_path, adam_config=config['adam_config'], patience=config['patience'])

Dataset splits sizes:  train 60 dev 10 test 20


In [ ]:
trainer.resume_training(config['num_epochs'])

Epoch 44, Step 1, Loss: 0.002372744493186474
Epoch 44, Step 2, Loss: 0.0023271767422556877
Epoch 44, Step 3, Loss: 0.0023334056604653597
Epoch 44, Step 4, Loss: 0.0023941288236528635
Epoch 44, Step 5, Loss: 0.0024161171168088913
Epoch 44, Step 6, Loss: 0.002338219666853547
Epoch 44, Step 7, Loss: 0.0022309082560241222
Epoch 44, Step 8, Loss: 0.002268332289531827
Epoch 44, Step 9, Loss: 0.0022418340668082237
Epoch 44, Step 10, Loss: 0.0023802067153155804
Epoch 44, Step 11, Loss: 0.0022850711829960346
Epoch 44, Step 12, Loss: 0.00221804715692997
Epoch 44, Step 13, Loss: 0.0021991734392941
Epoch 44, Step 14, Loss: 0.002169219544157386
Epoch 44, Step 15, Loss: 0.002205796539783478
Epoch 44, Step 16, Loss: 0.002335520228371024
Epoch 44, Step 17, Loss: 0.002327550668269396
Epoch 44, Step 18, Loss: 0.0027176185976713896
Epoch 44, Step 19, Loss: 0.002658173907548189
Epoch 44, Step 20, Loss: 0.0026219459250569344
Epoch 44, Step 21, Loss: 0.0025568734854459763
Epoch 44, Step 22, Loss: 0.00255202

In [ ]:
trainer.restore_best_checkpoint()
trainer.validate('Validation')
trainer.validate('Test')

Validation Metric MRRs: 0.6673801458103733
Validation Metric MAPs: 0.28354526721121115
Test Metric MRRs: 0.6601163697022185
Test Metric MAPs: 0.2465421311087633


(0.6601163697022185, 0.2465421311087633)